# Structural Analysis of RosettaScript PDB Output
The goal of this notebook is to develop a general method for correlating structural variation in RosettaScript output (i.e. RMSD) with interesting factors (DDG scores, ensemble size, trial sizes, kT).<br>
<br>
<b>Things to think about:</b>
* How much backbone variation is required/useful?
* Input starting PDB, WT output PDBs, and mutant output PDBs
* All-vs-all RMSDs for any outputs (DDG WT, DDG mutants, minimization comparison structures)
* RMSD for 8A repack sphere (repacking) and overall structure

<b>Table of Contents:</b>
* [RMSD-Calculator + X angles](#RMSD-Calculator-+-X-angles)<br>
* [X Angle Parser](#X-Angle-Parser)<br>
* [Mutant PBD Scraper](#Mutant-PBD-Scraper)<br>


## RMSD Calculator + X angles

This script uses pyRMSD to perform pairwaise RMSD calculations 1) globally, 2) within a shell defined by the find_neighbors function, and 3) for individual point mutations as defined by the read_mutations_resfile function.

In [ ]:
from Bio.PDB import *
import Bio.PDB
import os
import pyRMSD
from pyRMSD.matrixHandler import MatrixHandler
import pyRMSD.RMSDCalculator
from pyRMSD.availableCalculators import availableCalculators
import prody
import numpy as np
import scipy.spatial.distance

#From Kyle's Finalize.py
def read_mutations_resfile(filenum_dir):
    resfile = os.path.join(filenum_dir, 'mutations_repack.resfile')
    mutations = []
    with open(resfile, 'r') as f:
        post_start = False
        for line in f:
            if post_start:
                line = line.strip()
                pdb_resnum, chain, pikaa, mut_res = line.split()
                mutations.append( [pdb_resnum, chain, pikaa, mut_res] )
            elif line.startswith('start'):
                post_start = True
    return mutations

#From Kyle's Finalize.py
def find_neighbors(filenum_dir, pdb_path, neighbor_distance = 8.0):
    mutations = read_mutations_resfile(filenum_dir)
    parser = PDBParser(PERMISSIVE=1)
    open_strct = parser.get_structure('Open', pdb_path)

    # There should only be one model in PDB file
    num_models = 0
    for model in open_strct.get_models():
        num_models += 1
    assert( num_models == 1 )

    chain_list = [chain.get_id() for chain in open_strct[0].get_chains()]
    neighbors = set()
    for mutation in mutations:
        res_id, chain_id, pikaa, mut_aa = mutation
        mut_chain = str(chain_id)
        try:
            mut_pos = int( res_id )
            mut_insertion_code = ' '
        except ValueError:
            mut_pos = int( res_id[:-1] )
            mut_insertion_code = res_id[-1]

        mut_residue = open_strct[0][mut_chain][(' ', mut_pos, mut_insertion_code)]
        for chain in chain_list:
            for residue in [res.get_id() for res in open_strct[0][chain].get_residues()]:
                try:
                    # Kyle note - might be good to do something else for consistency, since not all residues have CB
                    dist = mut_residue['CB'] - open_strct[0][chain][residue]['CB']
                    if dist < neighbor_distance:
                        neighbors.add( (residue, chain) )
                except KeyError:
                    try:
                        dist = mut_residue['CA'] - open_strct[0][chain][residue]['CA']
                        if dist < neighbor_distance:
                            neighbors.add( (residue, chain) )
                    except KeyError:
                        pass

    return neighbors

#Kyle's RMSDWrapper.py (Spread out al over the place now)
def rmsd(input_pdbs, pyrmsd_calc, coordinates):
    if type(coordinates) is dict:
        rmsd_list = []
        for mutres, coord_set in coordinates.iteritems():
            rmsd_matrix = pyRMSD.matrixHandler.MatrixHandler().createMatrix(coord_set, pyrmsd_calc)
            rmsd_list.append([mutres, scipy.spatial.distance.squareform( rmsd_matrix.get_data() )])
        return rmsd_list
    else:
        rmsd_matrix = pyRMSD.matrixHandler.MatrixHandler().createMatrix(coordinates, pyrmsd_calc)
        return [scipy.spatial.distance.squareform( rmsd_matrix.get_data() )]
    
def global_ca_rms(input_pdbs):
    coordinates = np.array( [prody.parsePDB(input_pdb).select('calpha').getCoords() for input_pdb in input_pdbs] )
    return coordinates    
    
def neighborhood_rms(neighbors, reference, input_pdbs):
    temp = []

    for input_pdb in input_pdbs:
        atom_list = []
        hv = prody.parsePDB(input_pdb).getHierView()
        res_list = sorted( [hv[neighbor[1], neighbor[0][1]] for neighbor in neighbors] )
        for res in res_list:
            for atom in res:
                if atom.getElement() != 'H':
                    print atom
                    atom_list.append(atom.getCoords())
                else:
                    pass
        temp.append(atom_list)
        
    coordinates = np.asarray(temp)
    return coordinates
        
#Calculates sidechain rmsd        
def mutant_rms(datadir, input_pdbs):
    mutations = read_mutations_resfile(datadir)
    mutation_dict = {}
    
    for mutation in mutations:
        temp = []
        temp_nparray = []
        
        for input_pdb in input_pdbs: 
            atom_list = []
            hv = prody.parsePDB(input_pdb).getHierView()
            res_list = [hv[mutation[1], int(mutation[0])]]
            for res in res_list:
                for atom in res:
                    if atom.getElement() != 'H':
                        atom_list.append(atom.getCoords())
                    else:
                        pass
            temp.append(atom_list)
        
        temp_nparray = np.asarray(temp)
        
        mutation_dict['%s%s' %(mutation[1], mutation[0])] = temp_nparray
    print mutation_dict.keys()
    return mutation_dict

def chi_angles(datadir, input_pdbs):
    mutations = read_mutations_resfile(datadir)
    
    #http://www.ccp14.ac.uk/ccp/web-mirrors/garlic/garlic/commands/dihedrals.html
    chi1_dict = {'ARG':['N','CA','CB','CG'],
                 'ASN':['N','CA','CB','CG'],
                 'ASP':['N','CA','CB','CG'],
                 'CYS':['N','CA','CB','SG'],
                 'GLN':['N','CA','CB','CG'],
                 'GLU':['N','CA','CB','CG'],
                 'HIS':['N','CA','CB','CG'],
                 'ILE':['N','CA','CB','CG1'],
                 'LEU':['N','CA','CB','CG'],
                 'LYS':['N','CA','CB','CG'],
                 'MET':['N','CA','CB','CG'],
                 'PHE':['N','CA','CB','CG'],
                 'PRO':['N','CA','CB','CG'],
                 'SER':['N','CA','CB','OG'],
                 'THR':['N','CA','CB','OG1'],
                 'TRP':['N','CA','CB','CG'],
                 'TYR':['N','CA','CB','CG'],
                 'VAL':['N','CA','CB','CG1']}
    chi2_dict = {'ARG':['CA','CB','CG','CD'],
                 'ASN':['CA','CB','CG','OD1'],
                 'ASP':['CA','CB','CG','OD1'],
                 'GLN':['CA','CB','CG','CD'],
                 'GLU':['CA','CB','CG','CD'],
                 'HIS':['CA','CB','CG','ND1'],
                 'ILE':['CA','CB','CG1','CD'],
                 'LEU':['CA','CB','CG','CD1'],
                 'LYS':['CA','CB','CG','CD'],
                 'MET':['CA','CB','CG','SD'],
                 'PHE':['CA','CB','CG','CD1'],
                 'PRO':['CA','CB','CG','CD'],
                 'TRP':['CA','CB','CG','CD1'],
                 'TYR':['CA','CB','CG','CD1']}
    
    xangles_dict = {}
    
    for mutation in mutations:
        if mutation[3] == 'A' or 'G':
            continue
        else:
            xangles_dict['%s%s' %(mutation[0], mutation[1])] = {}
            counter = 1
            for input_pdb in input_pdbs:
                p = prody.parsePDB(input_pdb)
                hv = p.getHierView()

                current_res = hv[mutation[1], int(mutation[0])]
                templist = []

                if current_res.getResname() == 'ALA' or 'GLY':
                    continue

                else:
                    if current_res.getResname() in chi1_dict.keys():
                        current_res.select('name %s' %chi1_dict[current_res.getResname()])
                        chi1 = calcDihedral(current_res.select('name %s' %chi1_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][3]))
                        templist.append(chi1[0])

                    if current_res.getResname() in chi2_dict.keys():
                        chi2 = calcDihedral(current_res.select('name %s' %chi2_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][3]))
                        templist.append(chi2[0])

                    xangles_dict['%s%s' %(mutation[0], mutation[1])]['%s' %counter] = sorted(templist)

                counter = counter + 1

            print xangles_dict
    return xangles_dict

#Action!!!
def main():
    os.chdir('/home/james.lucas/Rotation/DDGBenchmarks_Test/')
    
    #Define things
    datadir = 'TestJobs/data/59648/'
    reference = 'TestJobs/data/59648/1TM1_EI.pdb'
    outputdir = 'TestJobs/output/59648/'
    
    # Use CUDA for GPU calculations, if avialable
    if 'QCP_CUDA_MEM_CALCULATOR' in availableCalculators():
        pyrmsd_calc = 'QCP_CUDA_MEM_CALCULATOR'
    else:
        pyrmsd_calc = 'QCP_SERIAL_CALCULATOR'
    
    input_temp = []
    for i in os.listdir(outputdir):
        if i.endswith('.pdb'):
            input_temp.append(os.path.join(outputdir, i ))
    input_pdbs = sorted(input_temp)
    
    neighbors = find_neighbors(datadir, reference, 8)
    
    point_mutants = mutant_rms(datadir, input_pdbs)
    #neighborhood = neighborhood_rms(neighbors, reference, input_pdbs)
    #global_ca = global_ca_rms(input_pdbs)
    
    asdf = rmsd(input_pdbs, pyrmsd_calc, point_mutants)
    
    for i in asdf:
        print i
        
main()

## X Angle Parser

Calculates X1 and X2 angles for each point mutation in the Zemu dataset DDG Rosettascript protocol output. The chi_angles function has been added to the RMSD calculator, this separate cell was added so that I could work on this on my Macbook... I can't figure out how to get pyRMSD on my Macbook :/

In [ ]:
import os
import json
import prody
from Bio.PDB import *

#From Kyle's Finalize.py
def read_mutations_resfile(filenum_dir):
    resfile = os.path.join(filenum_dir, 'mutations_repack.resfile')
    mutations = []
    with open(resfile, 'r') as f:
        post_start = False
        for line in f:
            if post_start:
                line = line.strip()
                pdb_resnum, chain, pikaa, mut_res = line.split()
                mutations.append( [pdb_resnum, chain, pikaa, mut_res] )
            elif line.startswith('start'):
                post_start = True
    return mutations

#From Kyle's Finalize.py
def find_neighbors(filenum_dir, pdb_path, neighbor_distance = 8.0):
    mutations = read_mutations_resfile(filenum_dir)
    parser = Bio.PDB.PDBParser(PERMISSIVE=1)
    open_strct = parser.get_structure('Open', pdb_path)

    # There should only be one model in PDB file
    num_models = 0
    for model in open_strct.get_models():
        num_models += 1
    assert( num_models == 1 )

    chain_list = [chain.get_id() for chain in open_strct[0].get_chains()]
    neighbors = set()
    for mutation in mutations:
        res_id, chain_id, pikaa, mut_aa = mutation
        mut_chain = str(chain_id)
        try:
            mut_pos = int( res_id )
            mut_insertion_code = ' '
        except ValueError:
            mut_pos = int( res_id[:-1] )
            mut_insertion_code = res_id[-1]

        mut_residue = open_strct[0][mut_chain][(' ', mut_pos, mut_insertion_code)]
        for chain in chain_list:
            for residue in [res.get_id() for res in open_strct[0][chain].get_residues()]:
                try:
                    # Kyle note - might be good to do something else for consistency, since not all residues have CB
                    dist = mut_residue['CB'] - open_strct[0][chain][residue]['CB']
                    if dist < neighbor_distance:
                        neighbors.add( (residue, chain) )
                except KeyError:
                    try:
                        dist = mut_residue['CA'] - open_strct[0][chain][residue]['CA']
                        if dist < neighbor_distance:
                            neighbors.add( (residue, chain) )
                    except KeyError:
                        pass

    return neighbors

def chi_angles(datadir, input_pdbs):
    mutations = read_mutations_resfile(datadir)
    
    #http://www.ccp14.ac.uk/ccp/web-mirrors/garlic/garlic/commands/dihedrals.html
    chi1_dict = {'ARG':['N','CA','CB','CG'],
                 'ASN':['N','CA','CB','CG'],
                 'ASP':['N','CA','CB','CG'],
                 'CYS':['N','CA','CB','SG'],
                 'GLN':['N','CA','CB','CG'],
                 'GLU':['N','CA','CB','CG'],
                 'HIS':['N','CA','CB','CG'],
                 'ILE':['N','CA','CB','CG1'],
                 'LEU':['N','CA','CB','CG'],
                 'LYS':['N','CA','CB','CG'],
                 'MET':['N','CA','CB','CG'],
                 'PHE':['N','CA','CB','CG'],
                 'PRO':['N','CA','CB','CG'],
                 'SER':['N','CA','CB','OG'],
                 'THR':['N','CA','CB','OG1'],
                 'TRP':['N','CA','CB','CG'],
                 'TYR':['N','CA','CB','CG'],
                 'VAL':['N','CA','CB','CG1']}
    chi2_dict = {'ARG':['CA','CB','CG','CD'],
                 'ASN':['CA','CB','CG','OD1'],
                 'ASP':['CA','CB','CG','OD1'],
                 'GLN':['CA','CB','CG','CD'],
                 'GLU':['CA','CB','CG','CD'],
                 'HIS':['CA','CB','CG','ND1'],
                 'ILE':['CA','CB','CG1','CD'],
                 'LEU':['CA','CB','CG','CD1'],
                 'LYS':['CA','CB','CG','CD'],
                 'MET':['CA','CB','CG','SD'],
                 'PHE':['CA','CB','CG','CD1'],
                 'PRO':['CA','CB','CG','CD'],
                 'TRP':['CA','CB','CG','CD1'],
                 'TYR':['CA','CB','CG','CD1']}
    
    xangles_dict = {}
    
    for mutation in mutations:
        if mutation[3] == 'A' or 'G':
            continue
        else:
            xangles_dict['%s%s' %(mutation[0], mutation[1])] = {}
            counter = 1
            for input_pdb in input_pdbs:
                p = prody.parsePDB(input_pdb)
                hv = p.getHierView()

                current_res = hv[mutation[1], int(mutation[0])]
                templist = []

                if current_res.getResname() == 'ALA' or 'GLY':
                    continue

                else:
                    if current_res.getResname() in chi1_dict.keys():
                        current_res.select('name %s' %chi1_dict[current_res.getResname()])
                        chi1 = calcDihedral(current_res.select('name %s' %chi1_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][3]))
                        templist.append(chi1[0])

                    if current_res.getResname() in chi2_dict.keys():
                        chi2 = calcDihedral(current_res.select('name %s' %chi2_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][3]))
                        templist.append(chi2[0])

                    xangles_dict['%s%s' %(mutation[0], mutation[1])]['%s' %counter] = sorted(templist)

                counter = counter + 1

            print xangles_dict

def main():
    os.chdir('/home/james.lucas/Rotation/DDGBenchmarks_Test/')
    #os.chdir('/Users/jameslucas/Kortemme_Rotation')
    #Define things
    datadir = 'TestJobs/data/59648/'
    reference = 'TestJobs/data/59648/1TM1_EI.pdb'
    outputdir = 'TestJobs/output/59648/'
    
    input_temp = []
    for i in os.listdir(outputdir):
        if i.endswith('.pdb'):
            input_temp.append(os.path.join(outputdir, i ))
    input_pdbs = sorted(input_temp)
    
    neighbors = find_neighbors(datadir, reference, 8)
    
    chi_angles(datadir, input_pdbs)
    
main()

## Mutant PBD Scraper
This script will take the sequence for each PDB in a subdirectory within some_ZeMu_dataset/data/ and modify the sequence based on the associated resfile. The script will then use prody_blast to find and download relevant structures from the PDB.

In [109]:
import os
import prody
import re
import requests
import klab.bio.rcsb as rcsb
import klab.bio.alignment as align
import klab.bio.pdb as PDB
import pandas as pd

#From Kyle's Finalize.py
def read_mutations_resfile(filenum_dir):
    resfile = os.path.join(filenum_dir, 'mutations_repack.resfile')
    mutations = []
    with open(resfile, 'r') as f:
        post_start = False
        for line in f:
            if post_start:
                line = line.strip()
                pdb_resnum, chain, pikaa, mut_res = line.split()
                mutations.append( [pdb_resnum, chain, pikaa, mut_res] )
            elif line.startswith('start'):
                post_start = True
    return mutations

def generate_mut_seq(pdbdir, hv):
    blast_dict = {}
    mutation_dict = {}
        
    mutations = read_mutations_resfile(pdbdir)
        
    for mutation in mutations:
        mutation_dict[mutation[0] + mutation[1]] = mutation[3]
    
    #Dictionary: 3- to 1-letter code
    res_dict = {
        'ALA':'A',
        'CYS':'C',
        'ASP':'D',
        'GLU':'E',
        'PHE':'F',
        'GLY':'G',
        'HIS':'H',
        'ILE':'I',
        'LYS':'K',
        'LEU':'L',
        'MET':'M',
        'ASN':'N',
        'PRO':'P',
        'GLN':'Q',
        'ARG':'R',
        'SER':'S',
        'THR':'T',
        'VAL':'V',
        'TRP':'W',
        'TYR':'Y'
    }
    
    for chain in hv:
        chain_string = ''
        for residue in chain:
            current_res =  str(residue.getResnum()) + str(residue.getChid())
            if current_res not in mutation_dict.keys():
                chain_string = chain_string + res_dict[residue.getResname()]
            else:
                chain_string = chain_string + mutation_dict[str(residue.getResnum()) + str(residue.getChid())]
        blast_dict[chain.getChid()] = chain_string
    
    return blast_dict

def get_PDBs(pdbdir):
    skip_blast = False
    
    for pdb_file in os.listdir(pdbdir):
        if pdb_file.endswith('.pdb'):
            actual_pdb_file = pdb_file
            output_dir = os.path.join('/kortemmelab/home/james.lucas/Mutant_PDBs', pdb_file[:-4])
            try:
                os.makedirs(output_dir)        
            except:
                print 'File directory %s already exists!!!' % pdb_file[:-4]
                
            pdb_chains = re.split(r'_', pdb_file)[1][:-4]
            pdb_id = re.split(r'_', pdb_file)[0]
            hv = prody.parsePDB(os.path.join(pdbdir, pdb_file)).getHierView()
            
            blast_dict = generate_mut_seq(pdbdir, hv)
            hit_list = []
            for sequence in blast_dict:
                set_dict = {}
                temp_set = set()
                blast_me = prody.blastPDB(blast_dict[sequence])
                hits_dict = blast_me.getHits(percent_identity=95)
                for hit in hits_dict:
                    temp_set.add(hit)

                hit_list.append(temp_set)

            common_set = hit_list[0]

            for entry in hit_list[1:]:
                common_set.intersection_update(entry)

            for hit in common_set:
                rcsb.download_pdb(hit, output_dir)
            
    return actual_pdb_file, common_set

def alignments(pdb_file, common_set, pdbdir):
    for pdb_id in common_set:
        clustal(pdb_file, pdb_id, pdbdir)

#Shane's ClustalO code
def clustal(pdb_WT, pdb_target, pdbdir):
    import pprint
    from klab.bio.clustalo import SequenceAligner
    from klab.bio.uniprot import UniParcEntry
    from klab.bio.basics import SubstitutionScore, PDBResidue
    from klab import colortext
    
    symbols = SubstitutionScore.clustal_symbols
    
    wt_pdb_path = os.path.join('/kortemmelab', 'home', 'james.lucas', '160322-james-backrub-rscript-full', 'data', pdbdir, 
                                            '{0}.pdb'.format(pdb_WT[:-4]))

    p1 = PDB.PDB.from_filepath(wt_pdb_path)
    p2 = PDB.PDB.from_filepath(os.path.join('/kortemmelab', 'home', 'james.lucas', 'Mutant_PDBs', pdb_WT[:-4], '{0}.pdb'.format(pdb_target)))

    sa = SequenceAligner()
    for chain_id, seq in (p1.seqres_sequences or p1.atom_sequences).iteritems():
        sa.add_sequence('{0}_{1}'.format(pdb_WT[:-4], chain_id), str(seq))
    for chain_id, seq in p2.seqres_sequences.iteritems():
        sa.add_sequence('{0}_{1}'.format(pdb_target, chain_id), str(seq))
    sa.align()
    
    # Prints the raw ClustalO output (for debugging)
    #colortext.message('\nClustalO output')
    #print(sa.alignment_output)

    # Find all differing residues
                    
    for chain_id_WT, seq_WT in (p1.seqres_sequences or p1.atom_sequences).iteritems():
        best_match_dict = sa.get_best_matches_by_id( '%s_%s' %(pdb_WT[:-4], chain_id_WT), cut_off = 90.0)
        
        for best_match_pdb in best_match_dict.keys():
            sa2 = SequenceAligner()
            sa2.add_sequence('{0}_{1}'.format(pdb_WT[:-4], chain_id_WT), str(seq_WT))
            sa2.add_sequence('{0}_{1}'.format(best_match_pdb[:-2], best_match_pdb[-1:]), str(p2.seqres_sequences[best_match_pdb[-1:]]))
            asddfasdf=p2.seqres_sequences[best_match_pdb[-1:]]
            sa2.align()
            #colortext.warning('Differing residues')
            #print('{0}_{1} -> {2}'.format(pdb_WT[:-4],chain_id_WT, best_match_pdb))
            resmap, clustal_matches = sa2.get_residue_mapping()
            
            #print clustal_matches
            raw_mut_list = []
            for first_idx, v in sorted(clustal_matches.iteritems()):
                # first_idx is an index into the first sequence, v is a SubstitutionScore object
                if symbols[v.clustal] != '*':
                    # get the index into the second sequence
                    second_idx = resmap[first_idx]

                    pdb_res1 = seq_WT.sequence[seq_WT.order[first_idx - 1]] # low-level access - there is probably a utility function to do this but I was hurrying
                    pdb_res2 = asddfasdf.sequence[asddfasdf.order[second_idx - 1]]
                    #print('  {0} -> {1}'.format(super(PDBResidue, pdb_res1).__repr__(), pdb_res2))
                    
                    raw_mut_list.append('  {0} -> {1}'.format(super(PDBResidue, pdb_res1).__repr__(), pdb_res2))
            mut_list_interpreter(raw_mut_list, pdb_WT, pdb_target)

#Takes list of mutation strings and converts it into mutations format
#If there is a 100% match in df, add remark in a new column with matched PDBID
def mut_list_interpreter(formatted_mut_list, pdb_WT, pdb_target):
    mut_string_temp = ''
    for element in formatted_mut_list:
        split_input = re.split(':| ', element.strip())
        residue_position = '    ' + split_input[1]
        mut_string_temp = mut_string_temp + '%s %s%s %s; ' %(split_input[0], split_input[2], residue_position[-4:], split_input[6])
    mut_string = mut_string_temp[:-2]
    
    output = open('/kortemmelab/home/james.lucas/Mutant_PDBs/Resfile_Mutations_Completed.txt', 'a')
    df = pd.read_csv('/kortemmelab/home/james.lucas/Mutant_PDBs/Resfile_Mutations.csv')
    for index, row in df.iterrows():
        if row[1].strip()== mut_string: 
            if pdb_WT[:4] == row[2].strip():
                output.write('%s,%s,%s,%s\n' % (row[0], pdb_WT, row[1], pdb_target) )
                print '%s,%s,%s,%s\n' % (row[0], pdb_WT, row[1], pdb_target)
            
def main():
    os.chdir('/kortemmelab/home/james.lucas/160322-james-backrub-rscript-full/data/')
    cwd = os.getcwd()
    output = open('/kortemmelab/home/james.lucas/Mutant_PDBs/Resfile_Mutations_Completed.txt', 'w')
    output.write('DatasetID,Mutations,PDBFileID,Matches\n')
    for pdbdir in os.listdir(cwd):
        if os.path.isdir(pdbdir):
            pdb_file, common_set = get_PDBs(pdbdir)
            alignments(pdb_file, common_set, pdbdir)
                
main()

576,1EMV_AB.pdb,B F  86 A,2vlq

572,1EMV_AB.pdb,B N  75 A,2vln

579,1EMV_AB.pdb,B K  97 A,2vlo

569,1EMV_AB.pdb,B R  54 A,2vlp

1125,1EMV_AB.pdb,A D  51 A,2gyk

1125,1EMV_AB.pdb,A D  51 A,2gyk

1125,1EMV_AB.pdb,A D  51 A,2gzj

1125,1EMV_AB.pdb,A D  51 A,2gzj

1111,1EMV_AB.pdb,A V  34 A,2gzi

1129,1EMV_AB.pdb,A Y  55 A,2gze

1116,1EMV_AB.pdb,A E  41 A,1fr2



In [1]:
import klab.bio.alignment as asdf
import klab.bio.pdb as PDB


a = set(['a', 'b', 'c', 'd'])
b = set(['e', 'b', 'c', 'd'])
c = set(['e', 'f', 'c', 'd'])

list = [a,b,c]
common_set = a

for i in list[1:]:
    common_set.intersection_update(i)

print common_set

os.getcwd()

asdf = open('59648/1TM1_EI.pdb')
pdb_strings = []
for line in asdf:
    pdb_strings.append(line.strip())

mypdb = PDB.PDB.from_filepath('59648/1TM1_EI.pdb')

print mypdb.pdb_id
print mypdb.seqres_sequences


set(['c', 'd'])


NameError: name 'os' is not defined